   1  : 1.58,
   0.1: 2.71,
  0.05: 3.84, 
  0.01: 6.63,
 0.005: 7.88, 
 0.001: 10.83

In [18]:
# 2.71 - 1.13

1.58

In [1]:
import pandas as pd
import numpy as np
from joblib import dump

from chi import square as cs

In [2]:
data = pd.read_excel("komentar/data_cleanU1.xlsx")
komentar = data.komentar.tolist()
label = np.array(data.label.tolist())
y = label
data.head()

,Column1,label,komentar
0,195,spam,ga nyesel deh aku order sana
1,480,non spam,wakakak tolol tulang tulang
2,539,non spam,sepatu keren kamu keren cara instan
3,564,spam,via q udh blja shoppe barang bagus kecewa mantap
4,596,non spam,sir what if use again kamisenglish that s just...


# Ekstraksi dan Seleksi Fitur

In [3]:
sf = cs.ChiSquare(alpha = 0.1)
sf.find_best_features(komentar, label, fitur = None)

In [4]:
print("jumlah fitur terbaik:",len(sf.best_features),"dari",len(sf.fitur), "fitur")
best_features = sf.best_features

jumlah fitur terbaik: 2749 dari 15136 fitur


# Pemodelan

## Pemodelan TF-IDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(vocabulary = best_features)#
X = vectorizer.fit_transform(komentar)
save = vectorizer.fit(komentar)
dump(save, "model/tfidf.w")
# print(vectorizer.get_feature_names())
fitur = vectorizer.get_feature_names()
print(X.shape)

(5047, 2749)


## Pemodelan SVM

In [6]:
from sklearn.svm import SVC
clf = SVC()#gamma='auto'

# Pemodelan dan Validasi Model

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

start = 0.1
stop = 1.5
step = start
alpha_list = np.arange(start,stop+step,step)
print(alpha_list)
akurasi = list()

ix = 0 #just counter
for alpha in alpha_list:
    clf.set_params(gamma=alpha)
    temp_akurasi = list()
    for train_index, test_index in skf.split(X, y):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        ##Trainin Model menggunakan sebagian data latih (K-fold)
        clf.fit(X_train,y_train)
        
        #prediksi menggunakan data uji dari k-fold
        y_pred = clf.predict(X_test)
        temp_akurasi.append(accuracy_score(y_test, y_pred))
    rerata_akurasi = sum(temp_akurasi)/len(temp_akurasi)
    akurasi.append(rerata_akurasi)
    ix +=1
    if ix%100 !=0:
        print(".", end="")
    else:
#         print(alpha, ' | ', rerata_akurasi)
        print("")

        pass

ditt = {
    "gamma":alpha_list,
    "akurasi":akurasi
}
index = akurasi.index(max(akurasi))
acc = pd.DataFrame.from_dict(ditt)
acc

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5]
...............

,gamma,akurasi
0,0.1,0.969299
1,0.2,0.973262
2,0.3,0.974648
3,0.4,0.975044
4,0.5,0.975242
5,0.6,0.974846
6,0.7,0.975242
7,0.8,0.975440
8,0.9,0.975440
9,1.0,0.975440


In [11]:
print(alpha_list[index])
print(akurasi[index])

1.2000000000000002
0.9754408297972654


In [12]:
print(alpha_list[index])
print(akurasi[index])

1.2000000000000002
0.9754408297972654


# Menyimpan Model

In [13]:
clf.set_params(gamma=alpha_list[index])

#Trainin Model menggunakan seluruh data latih
clf.fit(X, y)
dump(clf, "model/svm.m")

['model/svm.m']